In [10]:
lstm_cells.get_initial_state(batch_size=1, dtype=tf.float32)

[<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 128), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [13]:
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

batch_size = 3
sentence_max_length = 5
n_features = 2
new_shape = (batch_size, sentence_max_length, n_features)
x = tf.constant(np.reshape(np.arange(30), new_shape), dtype = tf.float32)

lstm_cells = tf.keras.layers.LSTMCell(128)

test_input = np.ones((2,128))

states = lstm_cells.get_initial_state(batch_size=1, dtype=tf.float32)
test_output = lstm_cells(test_input, states)
print("test_output: ", test_output)

#rnn_cells = [tf.keras.layers.LSTMCell(128) for _ in range(1)]
#stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)

#initial_state = stacked_lstm.get_initial_state(batch_size=4, dtype=tf.float32)
#initial_state
#lstm_layer = tf.keras.layers.RNN(stacked_lstm)

#result = lstm_layer(x)

test_output:  (<tf.Tensor: shape=(2, 128), dtype=float32, numpy=
array([[ 1.02849394e-01, -3.64125073e-02, -7.51350671e-02,
        -1.42729133e-01,  1.30299643e-01,  3.64783928e-02,
         2.49019876e-01, -5.94914258e-02,  9.30734817e-03,
        -1.45292446e-01,  1.03589945e-01, -1.29420519e-01,
        -8.62171873e-02, -1.51720047e-01, -1.13675460e-01,
        -2.35854089e-01,  2.03298237e-02, -1.40412590e-02,
        -3.29783529e-01, -7.47002615e-03, -1.54086679e-01,
        -1.79020494e-01,  5.61962873e-02,  1.94981415e-02,
        -7.13001266e-02, -2.26844057e-01, -1.43006757e-01,
        -9.67420191e-02, -3.61254299e-03, -1.55159831e-01,
         8.03648606e-02, -2.27837078e-02, -7.00157583e-02,
         2.87210764e-06,  1.35928199e-01,  3.16844927e-03,
        -9.63403359e-02, -5.82029484e-02, -6.96222112e-02,
         2.35813454e-01,  1.81506783e-01,  2.62233261e-02,
         5.87270297e-02, -1.40106037e-01,  2.51356177e-02,
        -1.27384260e-01, -1.21019334e-02,  1.69247

In [ ]:
import os
import random
import gym
import pylab
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import cv2
import threading
from threading import Thread, Lock
import time
import tensorflow_probability as tfp
from typing import Any, List, Sequence, Tuple

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

tfd = tfp.distributions


class OurModel(tf.keras.Model):
    def __init__(self, input_shape, action_space):
        super(OurModel, self).__init__()
        
        self.dense_0 = Dense(512, activation='relu')
        
        self.core = tf.keras.layers.LSTMCell(128)
        
        self.dense_1 = Dense(action_space)
        self.dense_2 = Dense(1)
    
    def initial_state(self, batch_size):
        return self.core.get_initial_state(batch_size=batch_size, dtype=tf.float32) 
    
    def call(self, X_input, core_state):
        # X_input.shape:  (1, 4, 80, 80)
        
        batch_size = X_input.shape[0]
        #print("batch_size: ", batch_size)
        #print("core_state: ", core_state)
        
        core_output_list = []
        for i in tf.range(0, batch_size):
            #X_input.shape:  (1, 4, 80, 80)
            #batch_size:  1
            #X_input[i].shape:  (4, 80, 80)
            #Input.shape:  (4, 512)
            #print("X_input[i].shape: ", X_input[i].shape)
            
            Input = tf.expand_dims(X_input[i], 0)
            
            Input = Flatten()(Input)
            #print("Input.shape 1: ", Input.shape)
            
            Input = self.dense_0(Input)
            #print("Input.shape 2: ", Input.shape)
            
            core_output, core_state = self.core(Input, core_state)
            #core_output_flattened = Flatten()(core_output)
            #print("core_output.shape: ", core_output.shape)
            core_output_list.append(core_output)
        
        outputs = tf.stack(core_output_list)
        #print("outputs.shape: ", outputs.shape)
        
        Ouput = Flatten()(outputs)
        #print("Ouput.shape: ", Ouput.shape)
        
        #Ouput = self.dense_0(Ouput)
        action_logit = self.dense_1(Ouput)
        value = self.dense_2(Ouput)
        
        #print("core_state output: ", core_state)
        
        return action_logit, value, core_state


def safe_log(x):
  """Computes a safe logarithm which returns 0 if x is zero."""
  return tf.where(
      tf.math.equal(x, 0),
      tf.zeros_like(x),
      tf.math.log(tf.math.maximum(1e-12, x)))


def take_vector_elements(vectors, indices):
    """
    For a batch of vectors, take a single vector component
    out of each vector.
    Args:
      vectors: a [batch x dims] Tensor.
      indices: an int32 Tensor with `batch` entries.
    Returns:
      A Tensor with `batch` entries, one for each vector.
    """
    return tf.gather_nd(vectors, tf.stack([tf.range(tf.shape(vectors)[0]), indices], axis=1))


huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
sparse_ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
mse_loss = tf.keras.losses.MeanSquaredError()


class A3CAgent:
    # Actor-Critic Main Optimization Algorithm
    def __init__(self, env_name):
        # Initialization
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.action_size = self.env.action_space.n
        self.EPISODES, self.episode, self.max_average = 2000000, 0, -21.0 # specific for pong
        self.lock = Lock()
        self.lr = 0.0001

        self.ROWS = 80
        self.COLS = 80
        self.REM_STEP = 4

        # Instantiate plot memory
        self.scores, self.episodes, self.average = [], [], []

        self.Save_Path = 'Models'
        self.state_size = (self.REM_STEP, self.ROWS, self.COLS)
        
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.path = '{}_A3C_{}'.format(self.env_name, self.lr)
        self.model_name = os.path.join(self.Save_Path, self.path)

        # Create Actor-Critic network model
        self.ActorCritic = OurModel(input_shape=self.state_size, action_space=self.action_size)
        
        self.optimizer = tf.keras.optimizers.Adam(self.lr)

    def act(self, state, core_state):
        # Use the network to predict the next action to take, using the model
        prediction = self.ActorCritic(state, core_state, training=False)
        action = tf.random.categorical(prediction[0], 1).numpy()

        return action[0][0], prediction[2]

    def discount_rewards(self, reward, dones):
        # Compute the gamma-discounted rewards over an episode
        gamma = 0.99    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0, len(reward))):
            if reward[i] != 0: # reset the sum, since this was a game boundary (pong specific!)
                running_add = 0

            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add

        discounted_r -= np.mean(discounted_r) # normalizing the result
        discounted_r /= np.std(discounted_r) # divide by standard deviation

        return discounted_r
        
    def replay(self, states, actions, rewards, dones):
        # reshape memory to appropriate shape for training
        states = np.vstack(states)
        
        # Compute discounted rewards
        discounted_r = self.discount_rewards(rewards, dones)
        discounted_r_ = np.vstack(discounted_r)
        with tf.GradientTape() as tape:
            core_state = self.ActorCritic.initial_state(1)
            prediction = self.ActorCritic(states, core_state, training=True)
            action_logits = prediction[0]
            values = prediction[1]
            
            action_logits_selected = take_vector_elements(action_logits, actions)
            
            advantages = discounted_r - np.stack(values)[:, 0] 
            
            action_logits_selected = tf.nn.softmax(action_logits_selected)
            action_logits_selected_probs = tf.math.log(action_logits_selected)
            
            action_logits_ = tf.nn.softmax(action_logits)
            dist = tfd.Categorical(probs=action_logits_)
            action_log_prob = dist.prob(actions)
            action_log_prob = tf.math.log(action_log_prob)
            
            actor_loss = -tf.math.reduce_mean(action_logits_selected_probs * advantages) 
            
            action_probs = tf.nn.softmax(action_logits)
            
            critic_loss_ = huber_loss(values, discounted_r)
            critic_loss = mse_loss(values, discounted_r_)
            critic_loss = tf.cast(critic_loss, 'float32')
            
            total_loss = actor_loss + critic_loss
        
        #print("total_loss: ", total_loss)
        #print("")
            
        grads = tape.gradient(total_loss, self.ActorCritic.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.ActorCritic.trainable_variables))
        
    def load(self, model_name):
        self.ActorCritic = load_model(model_name, compile=False)

    def save(self):
        self.ActorCritic.save(self.model_name)

    pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        if str(episode)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes, self.scores, 'b')
            pylab.plot(self.episodes, self.average, 'r')
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.savefig(self.path + ".png")
            except OSError:
                pass

        return self.average[-1]
    
    def imshow(self, image, rem_step=0):
        cv2.imshow(self.model_name + str(rem_step), image[rem_step,...])
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            return

    def GetImage(self, frame, image_memory):
        if image_memory.shape == (1,*self.state_size):
            image_memory = np.squeeze(image_memory)
            
        # croping frame to 80x80 size
        frame_cropped = frame[35:195:2, ::2,:]
        if frame_cropped.shape[0] != self.COLS or frame_cropped.shape[1] != self.ROWS:
            # OpenCV resize function 
            frame_cropped = cv2.resize(frame, (self.COLS, self.ROWS), interpolation=cv2.INTER_CUBIC)
        
        # converting to RGB (numpy way)
        frame_rgb = 0.299*frame_cropped[:,:,0] + 0.587*frame_cropped[:,:,1] + 0.114*frame_cropped[:,:,2]

        # convert everything to black and white (agent will train faster)
        frame_rgb[frame_rgb < 100] = 0
        frame_rgb[frame_rgb >= 100] = 255
        
        # dividing by 255 we expresses value to 0-1 representation
        new_frame = np.array(frame_rgb).astype(np.float32) / 255.0

        # push our data by 1 frame, similar as deq() function work
        #image_memory = np.roll(image_memory, 1, axis = 0)

        # inserting new frame to free space
        #image_memory[0,:,:] = new_frame

        #return np.expand_dims(image_memory, axis=0)
        return np.expand_dims(new_frame, axis=0)

    def reset(self, env):
        image_memory = np.zeros(self.state_size)
        frame = env.reset()
        #for i in range(self.REM_STEP):
        #    state = self.GetImage(frame, image_memory)
        state = self.GetImage(frame, image_memory)
            
        return state
    
    def step(self, action, env, image_memory):
        next_frame, reward, done, info = env.step(action)
        next_state = self.GetImage(next_frame, image_memory)
        
        return next_state, reward, done, info
    
    def train(self, n_threads):
        self.env.close()
        # Instantiate one environment per thread
        envs = [gym.make(self.env_name) for i in range(n_threads)]

        # Create threads
        threads = [threading.Thread(
                target=self.train_threading,
                daemon=True,
                args=(self,
                    envs[i],
                    i)) for i in range(n_threads)]

        for t in threads:
            time.sleep(2)
            t.start()
            
        for t in threads:
            time.sleep(10)
            t.join()
            
    def train_threading(self, agent, env, thread):
        while self.episode < self.EPISODES:
            # Reset episode
            score, done, SAVING = 0, False, ''
            state = self.reset(env)
            
            states, actions, rewards, dones = [], [], [], []
            core_state = self.ActorCritic.initial_state(1)
            while not done:
                action, core_state = agent.act(state, core_state)
                next_state, reward, done, _ = self.step(action, env, state)
                
                #print("next_state.shape: ", next_state.shape)
                #print("action: ", action)
                #print("reward: ", reward)
                #print("done: ", done)
                
                states.append(state)
                actions.append(action)
                rewards.append(reward)
                dones.append(done)

                score += reward
                state = next_state
                    
            self.lock.acquire()
            self.replay(states, actions, rewards, dones)
            self.lock.release()
            
            states, actions, rewards, dones = [], [], [], []
                    
            # Update episode count
            with self.lock:
                average = self.PlotModel(score, self.episode)
                # saving best models
                if average >= self.max_average:
                    self.max_average = average
                    #self.save()
                    SAVING = "SAVING"
                else:
                    SAVING = ""

                print("episode: {}/{}, thread: {}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, thread, score, average, SAVING))
                if(self.episode < self.EPISODES):
                    self.episode += 1

        env.close()            

    def test(self, Actor_name, Critic_name):
        self.load(Actor_name, Critic_name)
        for e in range(100):
            state = self.reset(self.env)
            done = False
            score = 0
            while not done:
                self.env.render()
                action = np.argmax(self.Actor.predict(state))
                state, reward, done, _ = self.step(action, self.env, state)
                score += reward
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
                    break

        self.env.close()


if __name__ == "__main__":
    env_name = 'PongDeterministic-v4'
    #env_name = 'Pong-v0'
    agent = A3CAgent(env_name)
    
    #agent.run() # use as A2C
    agent.train(n_threads=1) # use as A3C
    #agent.test('Models/Pong-v0_A3C_2.5e-05_Actor.h5', '')

episode: 0/2000000, thread: 0, score: -21.0, average: -21.00 SAVING
episode: 1/2000000, thread: 0, score: -21.0, average: -21.00 SAVING
episode: 2/2000000, thread: 0, score: -19.0, average: -20.33 SAVING
episode: 3/2000000, thread: 0, score: -21.0, average: -20.50 
episode: 4/2000000, thread: 0, score: -21.0, average: -20.60 
episode: 5/2000000, thread: 0, score: -21.0, average: -20.67 
episode: 6/2000000, thread: 0, score: -21.0, average: -20.71 
episode: 7/2000000, thread: 0, score: -18.0, average: -20.38 
episode: 8/2000000, thread: 0, score: -20.0, average: -20.33 SAVING
episode: 9/2000000, thread: 0, score: -20.0, average: -20.30 SAVING
episode: 10/2000000, thread: 0, score: -21.0, average: -20.36 
episode: 11/2000000, thread: 0, score: -21.0, average: -20.42 
episode: 12/2000000, thread: 0, score: -21.0, average: -20.46 
episode: 13/2000000, thread: 0, score: -19.0, average: -20.36 
episode: 14/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 15/2000000, thread: 0, sco

episode: 129/2000000, thread: 0, score: -20.0, average: -20.40 
episode: 130/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 131/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 132/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 133/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 134/2000000, thread: 0, score: -20.0, average: -20.38 
episode: 135/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 136/2000000, thread: 0, score: -21.0, average: -20.42 
episode: 137/2000000, thread: 0, score: -21.0, average: -20.42 
episode: 138/2000000, thread: 0, score: -21.0, average: -20.42 
episode: 139/2000000, thread: 0, score: -21.0, average: -20.42 
episode: 140/2000000, thread: 0, score: -21.0, average: -20.44 
episode: 141/2000000, thread: 0, score: -20.0, average: -20.42 
episode: 142/2000000, thread: 0, score: -20.0, average: -20.40 
episode: 143/2000000, thread: 0, score: -19.0, average: -20.36 
episode: 144/2000000, thread: 0, score: 

episode: 255/2000000, thread: 0, score: -20.0, average: -20.02 
episode: 256/2000000, thread: 0, score: -21.0, average: -20.02 
episode: 257/2000000, thread: 0, score: -19.0, average: -19.98 SAVING
episode: 258/2000000, thread: 0, score: -20.0, average: -20.02 
episode: 259/2000000, thread: 0, score: -21.0, average: -20.02 
episode: 260/2000000, thread: 0, score: -20.0, average: -20.02 
episode: 261/2000000, thread: 0, score: -20.0, average: -20.00 
episode: 262/2000000, thread: 0, score: -19.0, average: -19.96 SAVING
episode: 263/2000000, thread: 0, score: -21.0, average: -19.98 
episode: 264/2000000, thread: 0, score: -18.0, average: -19.94 SAVING
episode: 265/2000000, thread: 0, score: -20.0, average: -19.94 SAVING
episode: 266/2000000, thread: 0, score: -19.0, average: -19.92 SAVING
episode: 267/2000000, thread: 0, score: -20.0, average: -19.92 SAVING
episode: 268/2000000, thread: 0, score: -19.0, average: -19.88 SAVING
episode: 269/2000000, thread: 0, score: -19.0, average: -19.88